In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
# import glove nitin commented as its not used
# from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

# from keras.optimizers import Adam # nitin commented as Adam has been shifted to optimizer_v1 module.
from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
#from keras.backend.tensorflow_backend import set_session, clear_session, get_session # nitin commented as tensorflow_backend not used anymore
from keras.backend import set_session, clear_session, get_session

import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Reset Keras Session
def reset_keras(model):
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

def make_prediction_timeseries(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_timeseries(predictions, probs, ground_truth, model_name, 
                problem_type, iteration, hidden_unit_size, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

        
    file_name = str(hidden_unit_size)+"-"+model_name+"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+".p"
    
    result_path = "results/"
    print('auc{0}, auprc{1}, acc{2}, F1{3} '.format(auc,auprc,acc,F1))
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)

In [3]:
def timeseries_model(layer_name, number_of_unit):
    K.clear_session()
    
    sequence_input = Input(shape=(24,104),  name = "timeseries_input")
    
    if layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)
    else:
        x = GRU(number_of_unit)(sequence_input)
    
    # nitin removed the tf.contrib with tf.compat.v1.estimator 
    
    #logits_regularizer = tf.compat.v1.estimator.layers.l2_regularizer(scale=0.01)
    # removed tf.compat.v1.estimator.layers.xavier_initializer() with glorot_normal
    logits_regularizer = tf.keras.regularizers.L2(0.01)
    
    sigmoid_pred = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.glorot_normal(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    
    model = Model(inputs=sequence_input, outputs=sigmoid_pred)
    
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [4]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")

In [5]:
epoch_num = 100
model_patience = 3
monitor_criteria = 'val_loss'
batch_size = 128

unit_sizes = [128, 256]
#unit_sizes = [256]
iter_num = 11
target_problems = ['mort_hosp', 'mort_icu']#, 'los_3', 'los_7']
layers = ["LSTM", "GRU"]
#layers = ["GRU"]
for each_layer in layers:
    print("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print("Hidden unit: ", each_unit_size)
        for iteration in range(1, iter_num):
            print("Iteration number: ", iteration)
            print("=============================")

            for each_problem in target_problems:
                print ("Problem type: ", each_problem)
                print ("__________________")


                early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                best_model_name = str(each_layer)+"-"+str(each_unit_size)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                    save_best_only=True, mode='min', period=1)


                callbacks = [early_stopping_monitor, checkpoint]

                model = timeseries_model(each_layer, each_unit_size)
                model.fit(x_train_lstm, y_train[each_problem], epochs=epoch_num, verbose=1, 
                          validation_data=(x_dev_lstm, y_dev[each_problem]), callbacks=callbacks, batch_size= batch_size)

                model.load_weights(best_model_name)

                probs, predictions = make_prediction_timeseries(model, x_test_lstm)
                save_scores_timeseries(predictions, probs, y_test[each_problem].values,str(each_layer),
                                       each_problem, iteration, each_unit_size,type_of_ner)
                reset_keras(model)
                #del model
                clear_session()
                gc.collect()

Layer:  LSTM
Hidden unit:  128
Iteration number:  1
Problem type:  mort_hosp
__________________
Epoch 1/100
115/115 [==============================] - ETA: 0s - loss: 0.3071 - acc: 0.8903
Epoch 00001: val_loss improved from inf to 0.25073, saving model to LSTM-128-mort_hosp-best_model.hdf5
115/115 [==============================] - 8s 51ms/step - loss: 0.3071 - acc: 0.8903 - val_loss: 0.2507 - val_acc: 0.9158
Epoch 2/100
114/115 [============================>.] - ETA: 0s - loss: 0.2377 - acc: 0.9161
Epoch 00002: val_loss improved from 0.25073 to 0.24871, saving model to LSTM-128-mort_hosp-best_model.hdf5
115/115 [==============================] - 7s 61ms/step - loss: 0.2375 - acc: 0.9163 - val_loss: 0.2487 - val_acc: 0.9077
Epoch 3/100
114/115 [============================>.] - ETA: 0s - loss: 0.2118 - acc: 0.9262
Epoch 00003: val_loss improved from 0.24871 to 0.24713, saving model to LSTM-128-mort_hosp-best_model.hdf5
115/115 [==============================] - 7s 61ms/step - loss: 0.2

Iteration number:  3
Problem type:  mort_hosp
__________________
Epoch 1/100
114/115 [============================>.] - ETA: 0s - loss: 0.3055 - acc: 0.8893- ETA: 1
Epoch 00001: val_loss improved from inf to 0.24889, saving model to LSTM-128-mort_hosp-best_model.hdf5
115/115 [==============================] - 9s 63ms/step - loss: 0.3054 - acc: 0.8893 - val_loss: 0.2489 - val_acc: 0.9110
Epoch 2/100
114/115 [============================>.] - ETA: 0s - loss: 0.2386 - acc: 0.9176
Epoch 00002: val_loss did not improve from 0.24889
115/115 [==============================] - 7s 60ms/step - loss: 0.2382 - acc: 0.9178 - val_loss: 0.2503 - val_acc: 0.9086
Epoch 3/100
114/115 [============================>.] - ETA: 0s - loss: 0.2167 - acc: 0.9247
Epoch 00003: val_loss did not improve from 0.24889
115/115 [==============================] - 7s 61ms/step - loss: 0.2163 - acc: 0.9248 - val_loss: 0.2523 - val_acc: 0.9033
Epoch 4/100
114/115 [============================>.] - ETA: 0s - loss: 0.1944 - 

115/115 [==============================] - ETA: 0s - loss: 0.2140 - acc: 0.9244
Epoch 00003: val_loss improved from 0.25089 to 0.24645, saving model to LSTM-128-mort_hosp-best_model.hdf5
115/115 [==============================] - 7s 62ms/step - loss: 0.2140 - acc: 0.9244 - val_loss: 0.2464 - val_acc: 0.9100
Epoch 4/100
114/115 [============================>.] - ETA: 0s - loss: 0.1937 - acc: 0.9330- ETA: 2s - loss:  - ETA: 0s - loss: 0.190
Epoch 00004: val_loss did not improve from 0.24645
115/115 [==============================] - 7s 60ms/step - loss: 0.1935 - acc: 0.9330 - val_loss: 0.2491 - val_acc: 0.9100
Epoch 5/100
114/115 [============================>.] - ETA: 0s - loss: 0.1722 - acc: 0.9399- ETA:  - ETA: 1s - loss: 0.
Epoch 00005: val_loss did not improve from 0.24645
115/115 [==============================] - 7s 58ms/step - loss: 0.1720 - acc: 0.9400 - val_loss: 0.2598 - val_acc: 0.9029
Epoch 6/100
115/115 [==============================] - ETA: 0s - loss: 0.1540 - acc: 0.9491

Epoch 3/100
114/115 [============================>.] - ETA: 0s - loss: 0.2147 - acc: 0.9238
Epoch 00003: val_loss did not improve from 0.24759
115/115 [==============================] - 6s 49ms/step - loss: 0.2144 - acc: 0.9240 - val_loss: 0.2524 - val_acc: 0.9067
Epoch 4/100
115/115 [==============================] - ETA: 0s - loss: 0.1947 - acc: 0.9322
Epoch 00004: val_loss did not improve from 0.24759
115/115 [==============================] - 6s 50ms/step - loss: 0.1947 - acc: 0.9322 - val_loss: 0.2558 - val_acc: 0.9010
Epoch 5/100
114/115 [============================>.] - ETA: 0s - loss: 0.1730 - acc: 0.9407
Epoch 00005: val_loss did not improve from 0.24759
115/115 [==============================] - 6s 51ms/step - loss: 0.1729 - acc: 0.9406 - val_loss: 0.2744 - val_acc: 0.9067
auc0.884195350492371, auprc0.5811189876309254, acc0.9103332534164469, F10.42638036809815955 
0.884195350492371 0.5811189876309254 0.9103332534164469 0.42638036809815955
Problem type:  mort_icu
____________

114/115 [============================>.] - ETA: 0s - loss: 0.1944 - acc: 0.9316- E - ETA: 1s - loss: 0.1922 
Epoch 00004: val_loss did not improve from 0.24285
115/115 [==============================] - 5s 46ms/step - loss: 0.1938 - acc: 0.9319 - val_loss: 0.2536 - val_acc: 0.9043
Epoch 5/100
115/115 [==============================] - ETA: 0s - loss: 0.1708 - acc: 0.9407
Epoch 00005: val_loss did not improve from 0.24285
115/115 [==============================] - 5s 45ms/step - loss: 0.1708 - acc: 0.9407 - val_loss: 0.2546 - val_acc: 0.8995
auc0.8775889678771788, auprc0.5633480790850149, acc0.9112922560537041, F10.40514469453376206 
0.8775889678771788 0.5633480790850149 0.9112922560537041 0.40514469453376206
Problem type:  mort_icu
__________________
Epoch 1/100
114/115 [============================>.] - ETA: 0s - loss: 0.2508 - acc: 0.9137
Epoch 00001: val_loss improved from inf to 0.18758, saving model to LSTM-128-mort_icu-best_model.hdf5
115/115 [==============================] - 8s

...

Iteration number:  3
Problem type:  mort_hosp
__________________
Epoch 1/100
115/115 [==============================] - ETA: 0s - loss: 0.3136 - acc: 0.8829
Epoch 00001: val_loss improved from inf to 0.24650, saving model to GRU-256-mort_hosp-best_model.hdf5
115/115 [==============================] - 15s 109ms/step - loss: 0.3136 - acc: 0.8829 - val_loss: 0.2465 - val_acc: 0.9134
Epoch 2/100
115/115 [==============================] - ETA: 0s - loss: 0.2475 - acc: 0.9117
Epoch 00002: val_loss improved from 0.24650 to 0.24635, saving model to GRU-256-mort_hosp-best_model.hdf5
115/115 [==============================] - 11s 98ms/step - loss: 0.2475 - acc: 0.9117 - val_loss: 0.2463 - val_acc: 0.9105
Epoch 3/100
115/115 [==============================] - ETA: 0s - loss: 0.2322 - acc: 0.9167
Epoch 00003: val_loss improved from 0.24635 to 0.23976, saving model to GRU-256-mort_hosp-best_model.hdf5
115/115 [==============================] - 12s 103ms/step - loss: 0.2322 - acc: 0.9167 - val_loss:

Epoch 1/100
115/115 [==============================] - ETA: 0s - loss: 0.2541 - acc: 0.9161- ETA: 0s - loss: 0.2588 - acc:
Epoch 00001: val_loss improved from inf to 0.18710, saving model to GRU-256-mort_icu-best_model.hdf5
115/115 [==============================] - 14s 99ms/step - loss: 0.2541 - acc: 0.9161 - val_loss: 0.1871 - val_acc: 0.9421
Epoch 2/100
115/115 [==============================] - ETA: 0s - loss: 0.1852 - acc: 0.9414
Epoch 00002: val_loss improved from 0.18710 to 0.18422, saving model to GRU-256-mort_icu-best_model.hdf5
115/115 [==============================] - 11s 94ms/step - loss: 0.1852 - acc: 0.9414 - val_loss: 0.1842 - val_acc: 0.9431
Epoch 3/100
115/115 [==============================] - ETA: 0s - loss: 0.1695 - acc: 0.9435
Epoch 00003: val_loss improved from 0.18422 to 0.17336, saving model to GRU-256-mort_icu-best_model.hdf5
115/115 [==============================] - 11s 98ms/step - loss: 0.1695 - acc: 0.9435 - val_loss: 0.1734 - val_acc: 0.9435
Epoch 4/100
1

115/115 [==============================] - ETA: 0s - loss: 0.2208 - acc: 0.9214
Epoch 00004: val_loss improved from 0.24437 to 0.24073, saving model to GRU-256-mort_hosp-best_model.hdf5
115/115 [==============================] - 12s 109ms/step - loss: 0.2208 - acc: 0.9214 - val_loss: 0.2407 - val_acc: 0.9158
Epoch 5/100
115/115 [==============================] - ETA: 0s - loss: 0.2088 - acc: 0.925 - ETA: 0s - loss: 0.2088 - acc: 0.9257
Epoch 00005: val_loss improved from 0.24073 to 0.23779, saving model to GRU-256-mort_hosp-best_model.hdf5
115/115 [==============================] - 12s 106ms/step - loss: 0.2088 - acc: 0.9257 - val_loss: 0.2378 - val_acc: 0.9115
Epoch 6/100
115/115 [==============================] - ETA: 0s - loss: 0.1953 - acc: 0.9284- ETA: 2s 
Epoch 00006: val_loss did not improve from 0.23779
115/115 [==============================] - 12s 105ms/step - loss: 0.1953 - acc: 0.9284 - val_loss: 0.2443 - val_acc: 0.9057
Epoch 7/100
115/115 [==============================] 

115/115 [==============================] - ETA: 0s - loss: 0.1365 - acc: 0.9542
Epoch 00006: val_loss did not improve from 0.16840
115/115 [==============================] - 12s 107ms/step - loss: 0.1365 - acc: 0.9542 - val_loss: 0.1745 - val_acc: 0.9431
Epoch 7/100
115/115 [==============================] - ETA: 0s - loss: 0.1273 - acc: 0.9565
Epoch 00007: val_loss did not improve from 0.16840
115/115 [==============================] - 12s 108ms/step - loss: 0.1273 - acc: 0.9565 - val_loss: 0.1792 - val_acc: 0.9373
Epoch 8/100
115/115 [==============================] - ETA: 0s - loss: 0.1112 - acc: 0.9645
Epoch 00008: val_loss did not improve from 0.16840
115/115 [==============================] - 12s 102ms/step - loss: 0.1112 - acc: 0.9645 - val_loss: 0.1894 - val_acc: 0.9383
auc0.8869672740640482, auprc0.5278341076157913, acc0.9400623351714217, F10.4444444444444444 
0.8869672740640482 0.5278341076157913 0.9400623351714217 0.4444444444444444
Iteration number:  8
Problem type:  mort_h

115/115 [==============================] - ETA: 0s - loss: 0.1655 - acc: 0.9407
Epoch 00008: val_loss did not improve from 0.23418
115/115 [==============================] - 12s 100ms/step - loss: 0.1655 - acc: 0.9407 - val_loss: 0.2541 - val_acc: 0.9072
auc0.8784479802603904, auprc0.5636257529615422, acc0.908175497482618, F10.46433566433566437 
0.8784479802603904 0.5636257529615422 0.908175497482618 0.46433566433566437
Problem type:  mort_icu
__________________
Epoch 1/100
115/115 [==============================] - ETA: 0s - loss: 0.2488 - acc: 0.9179
Epoch 00001: val_loss improved from inf to 0.18706, saving model to GRU-256-mort_icu-best_model.hdf5
115/115 [==============================] - 14s 107ms/step - loss: 0.2488 - acc: 0.9179 - val_loss: 0.1871 - val_acc: 0.9392
Epoch 2/100
115/115 [==============================] - ETA: 0s - loss: 0.1839 - acc: 0.9403
Epoch 00002: val_loss improved from 0.18706 to 0.17879, saving model to GRU-256-mort_icu-best_model.hdf5
115/115 [==========